<a href="https://colab.research.google.com/github/renya1993/competition_practice_/blob/main/%E3%82%BF%E3%82%A4%E3%82%BF%E3%83%8B%E3%83%83%E3%82%AF%E3%82%B3%E3%83%B3%E3%83%9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>